In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach FLowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
import scipy.stats as stats
import logging
import numpy as np
import time
import datetime
from tqdm import tqdm
import os
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import multiprocessing
import run_script

import pandas
from pathlib import Path
import subprocess
import sqlite3
from fnmatch import fnmatch

import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
from i2c_gui.chips.etroc2_chip import register_decoding

from notebooks.notebook_helpers import *

# Specify Global Info

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_name = "ET2_W42_IP7_2"
tag = "HV210V"
chip_figname = f"{chip_name}_LowBiasCurrent_{tag}"
chip_figtitle= chip_name+" LowBiasCurrent "+tag

# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_address = 0x62
ws_address = None

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

hostname = '192.168.2.7'

# Make i2c_connection class object for chip configuration

In [ ]:
i2c_conn = i2c_connection(port,[chip_address],[chip_name],[("1","1"),("1","1")])

# Config chips

### Basic config

In [ ]:
# 0-0-0-disable/default all pixels - auto TH CAL - set basic peripherals - peri reg check - pixel ID check
i2c_conn.config_chips('00100111')

### Only do auto TH cal and set basic peripherals

In [ ]:
# 0-0-0-disable/default all pixels - auto TH CAL - set basic peripherals - peri reg check - pixel ID check
i2c_conn.config_chips('00001100')

## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
BL_map_THCal,NW_map_THCal,BL_df = i2c_conn.get_auto_cal_maps(chip_address)
fig = plt.figure(dpi=200, figsize=(10,5))
gs = fig.add_gridspec(1,2)

ax0 = fig.add_subplot(gs[0,0])
ax0.set_title(f"{chip_figtitle}: BL (DAC LSB)", size=8)
img0 = ax0.imshow(BL_map_THCal, interpolation='none')
ax0.set_aspect("equal")
ax0.invert_xaxis()
ax0.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img0, cax=cax, orientation="vertical")

ax1 = fig.add_subplot(gs[0,1])
ax1.set_title(f"{chip_figtitle}: NW (DAC LSB)", size=8)
img1 = ax1.imshow(NW_map_THCal, interpolation='none')
ax1.set_aspect("equal")
ax1.invert_xaxis()
ax1.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img1, cax=cax, orientation="vertical")

for x in range(16):
    for y in range(16):
        # if(BL_map_THCal.T[x,y]==0): continue
        ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
        ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
plt.savefig(fig_path+"/"+chip_figname+"_BL_NW_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
plt.show()

# One time run to set fpga firmware and LEDs

In [ ]:
parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved -v -w -s 0x0000 -p 0x000b -d 0x1800 -a 0x0011 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Start_LEDs'))
process.start()

IPC_queue.put('start L1A trigger bit')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

# Define pixels of interest to scan

In [ ]:
# row_list = [15, 15, 15, 15, 14, 14, 14, 14]
# col_list = [6, 7, 8, 9, 6, 7, 8, 9]
row_list = [15]
col_list = [6]
scan_list = list(zip(col_list, row_list))
print(scan_list)

# Define pixels of interest for plotting

In [ ]:
# row_list = [15, 15, 15, 15, 14, 14, 14, 14]
# col_list = [6, 7, 8, 9, 6, 7, 8, 9]
row_list = [15]
col_list = [6]
plot_list = list(zip(col_list, row_list))
print(plot_list)

# Calibrate PLL

In [ ]:
print(f"Calibrating PLL for chip {hex(chip_address)} ")
chip = i2c_conn.get_chip_i2c_connection(chip_address)
i2c_conn.peripheral_decoded_register_write("asyPLLReset", "0", chip)
time.sleep(0.2)
i2c_conn.peripheral_decoded_register_write("asyPLLReset", "1", chip)
i2c_conn.peripheral_decoded_register_write("asyStartCalibration", "0", chip)
time.sleep(0.2)
i2c_conn.peripheral_decoded_register_write("asyStartCalibration", "1", chip)

# Run Noise-Only S-Curve

In [ ]:
pedestal_scan_step = 1
thresholds = np.arange(-20,20,pedestal_scan_step) # relative to BL
scan_name = chip_figname+"_VRef_SCurve_NoiseOnly"
fpga_time = 3

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

# Loop for enable/disable charge injection per pixel (single!!!)
for index, (col, row) in zip(range(len(row_list)), scan_list):  
    print("Pixel:",col,row)
    pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)

    # Ensure charge injection is disabled
    i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)

    # Bypass Cal Threshold
    i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)

    # start FPGA state machine
    threshold_name = scan_name+f'_Pixel_C{col}_R{row}'
    (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v -w --reset_till_trigger_linked -c 0x0001 --fpga_data_time_limit 3 --fpga_data --check_trigger_link_at_end --nodaq".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_noiseOnly'))
    process.start()
    process.join()
    
    for DAC in tqdm(thresholds[:], desc=f'DAC Loop for Pixel {col},{row}', leave=False):
        threshold = int(DAC+BL_map_THCal[row][col])

        # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
        i2c_conn.pixel_decoded_register_write("DAC", format(threshold, '010b'), pixel_connected_chip)
        # print("DAC:", threshold, ", Read TH:", i2c_conn.pixel_decoded_register_read("TH", "Status", pixel_connected_chip, need_int=True))

        (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v --reset_till_trigger_linked -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data --check_trigger_link_at_end --nodaq --DAC_Val {int(threshold)}".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_NoiseOnly_{threshold}'))
        process.start()
        process.join()
        
    # Disable charge injection
    i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)

## Plot the Noise-Only S-Curves

In [ ]:
root = '../ETROC-Data'
file_pattern = "*FPGA_Data.dat"
scan_name = chip_figname+"_VRef_SCurve_NoiseOnly"
triggerbit_full_Scurve = {row:{col:{thr+int(BL_map_THCal[row][col]):0 for thr in thresholds} for col in range(16)} for row in range(16)}
for index, (col, row) in zip(range(len(row_list)), plot_list):
    BL = int(BL_map_THCal[row][col])
    print(f'Pixel {col},{row}')
    path_pattern = f"*{today.isoformat()}_Array_Test_Results/{scan_name}_Pixel_C{col}_R{row}"
    file_list = []
    for path, subdirs, files in os.walk(root):
        if not fnmatch(path, path_pattern): continue
        for name in files:
            pass
            if fnmatch(name, file_pattern):
                file_list.append(os.path.join(path, name))
                print(file_list[-1])
    total_files = len(file_list)
    for file_index, file_name in enumerate(file_list):
        print(f"{file_index+1}/{total_files}")
        with open(file_name) as infile:
            for line in infile:
                text_list = line.split(',')
                FPGA_triggerbit = int(text_list[5])
                DAC = int(text_list[-1])
                if DAC == -1: continue
                try:
                    triggerbit_full_Scurve[row][col][DAC] += FPGA_triggerbit
                except:
                    print(f"Couldn't find DAC: {DAC} in file list")

In [ ]:
col_list, row_list = zip(*plot_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*5,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='r', ls='-', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='r', ls='-', lw=0.7)
        ax0.plot([thr+BL for thr in thresholds], triggerbit_full_Scurve[row][col].values(), '.-', color='b',lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        plt.legend(loc="upper right")
        # plt.yscale("log")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Noise Only S-Curve",size=8)
        plt.tight_layout()
plt.savefig(fig_path+"/"+chip_figname+"_NoiseOnly_S-Curve_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

# Qinj S Curve Scans

In [ ]:
# Full Charges
# QInjEns = [3, 5, 6, 8, 10, 12, 15, 17, 20, 22, 25, 27, 30]
# Recommend for TID
QInjEns = [5, 6, 8, 15, 22, 27]

## Peak DAC Value Binary Search

In [ ]:
DAC_scan_max = 1020
scan_name = chip_figname+"_VRef_SCurve_BinarySearch"
fpga_time = 3

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

# Loop for enable/disable charge injection per pixel (single!!!)
for index, (col, row) in zip(range(len(row_list)), scan_list):
    print("Pixel:",col,row)
    pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)
    # Enable charge injection
    i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("QInjEn", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)
    # Bypass Cal Threshold
    i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)
    for QInj in QInjEns:
        # Modifying charge injected
        i2c_conn.pixel_decoded_register_write("QSel", format(QInj, '05b'), pixel_connected_chip)
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'
        (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()

        a = BL_map_THCal[row][col]
        b = DAC_scan_max
        header_max = -1
        while b-a>1:
            DAC = int(np.floor((a+b)/2))
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            i2c_conn.pixel_decoded_register_write("DAC", format(DAC, '010b'), pixel_connected_chip)
            # print("QInj:", QInj, ", DAC:", DAC, ", Read TH:", i2c_conn.pixel_decoded_register_read("TH", "Status", pixel_connected_chip, need_int=True))

            (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_{QInj}_{DAC}'))
            process.start()
            process.join()
            
            continue_flag = False
            root = '../ETROC-Data'
            file_pattern = "*FPGA_Data.dat"
            path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            total_files = len(file_list)
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    lines = infile.readlines()
                    last_line = lines[-1]
                    first_line = lines[0]
                    header_max = int(first_line.split(',')[4])
                    text_list = last_line.split(',')
                    FPGA_state = text_list[0]
                    line_DAC = int(text_list[-1])
                    if(FPGA_state==0 or line_DAC!=DAC): 
                        continue_flag=True
                        continue
                    TDC_data = int(text_list[3])
                    # Condition handling for Binary Search
                    if(TDC_data>=header_max/2.):
                        a = DAC
                    else:
                        b = DAC                    
            if(continue_flag): continue  
            
    # Disable charge injection
    i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)

### Extract Peak DAC

In [ ]:
scan_name = chip_figname+"_VRef_SCurve_BinarySearch"
QInj_Peak_DAC_map = {row:{col:{q:0 for q in QInjEns} for col in range(16)} for row in range(16)}
for index, (col, row) in zip(range(len(row_list)), plot_list): 
    print("Pixel:",col,row)
    for QInj in tqdm(QInjEns, desc=f'Charge Loop for Pixel {col},{row}', leave=False):
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'
        path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
        file_list = []
        for path, subdirs, files in os.walk(root):
            if not fnmatch(path, path_pattern): continue
            for name in files:
                pass
                if fnmatch(name, file_pattern):
                    file_list.append(os.path.join(path, name))
        total_files = len(file_list)
        for file_index, file_name in enumerate(file_list):
            with open(file_name) as infile:
                last_line = infile.readlines()[-1]
                text_list = last_line.split(',')
                DAC = int(text_list[-1])
                QInj_Peak_DAC_map[row][col][QInj] = DAC

### Plot QInj vs DAC

In [ ]:
col_list, row_list = zip(*plot_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*7,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        BL = int(np.floor(BL_map_THCal[row][col]))
        NW = abs(int(np.floor(NW_map_THCal[row][col])))
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axhline(BL, color='k', lw=0.8, label=f"BL = {BL} DAC LSB")
        ax0.axhline(BL+NW, color='k',ls="--", lw=0.8, label=f"NW = $\pm${NW} DAC LSB")
        ax0.axhline(BL-NW, color='k',ls="--", lw=0.8)
        X = []
        Y = []
        for QInj in QInjEns:
            ax0.plot(QInj, QInj_DAC_map[row][col][QInj], 'rx')
            X.append(QInj)
            Y.append(QInj_Peak_DAC_map[row][col][QInj])
        X = np.array(X[:])
        Y = np.array(Y[:])
        (m, b), cov = np.polyfit(X, Y, 1, cov = True)
        n = Y.size
        Yfit = np.polyval((m,b), X)
        errorbars = np.sqrt(np.diag(cov))
        x_range = np.linspace(0, 35, 100)
        y_est = b + m*x_range
        resid = Y - Yfit
        s_err = np.sqrt(np.sum(resid**2)/(n - 2))
        t = stats.t.ppf(0.95, n - 2)
        ci2= t * s_err * np.sqrt(    1/n + (x_range - np.mean(X))**2/(np.sum((X)**2)-n*np.sum((np.mean(X))**2)))
        
        ax0.plot(x_range, y_est, 'b-', lw=-.8, label=f"DAC_TH = ({m:.3f}$\pm${errorbars[0]:.3f} [1/fC])$\cdot$Q + ({b:.3f}$\pm${errorbars[1]:.3f})")
        plt.fill_between(x_range, y_est+ci2, y_est-ci2, color='b',alpha=0.2, label="95% Confidence Interval on Linear Fit")
        ax0.set_xlabel("Charge Injected [fC]")
        ax0.set_ylabel("DAC Threshold [LSB]")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Qinj Sensitivity Plot",size=10)
        plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(fig_path+"/"+chip_figname+"_QInj_Sensitivity_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
plt.show()

## Full DAC scan using counter

In [ ]:
pedestal_scan_step = 2
scan_name = chip_figname+"_VRef_SCurve"
fpga_time = 3

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

col_list, row_list = zip(*scan_list)

# Loop for enable/disable charge injection per pixel (single!!!)
for index, (col, row) in zip(range(len(row_list)), scan_list):
    print("Pixel:",col,row)
    pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)
    # Enable charge injection
    i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("QInjEn", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)
    # Bypass Cal Threshold
    i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)
    for QInj in QInjEns:
        # Modifying charge injected
        i2c_conn.pixel_decoded_register_write("QSel", format(QInj, '05b'), pixel_connected_chip)
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'
        (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()

        thresholds = np.arange(BL_map_THCal[row][col]-20, QInj_Peak_DAC_map[row][col][QInj]+10, pedestal_scan_step)
        
        for DAC in tqdm(thresholds, desc=f'DAC Loop for Pixel ({col},{row}) & Charge {QInj} fC', leave=False):
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            i2c_conn.pixel_decoded_register_write("DAC", format(int(DAC), '010b'), pixel_connected_chip)
            (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Pixel_C{col}_R{row}_{QInj}_{int(DAC)}'))
            process.start()
            process.join()
            
    # Disable charge injection
    i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)
    i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)


### Plot Full Scan Output

In [ ]:
scan_name = chip_figname+"_VRef_SCurve"
root = '../ETROC-Data'
file_pattern = "*FPGA_Data.dat"
triggerbit_full_Scurve = {row:{col:{q:{} for q in QInjEns} for col in range(16)} for row in range(16)}
data_full_Scurve = {row:{col:{q:{} for q in QInjEns} for col in range(16)} for row in range(16)}
col_list, row_list = zip(*plot_list)
for index, (col, row) in zip(range(len(row_list)), plot_list): 
    BL = int(BL_map_THCal[row][col])
    for QInj in (QInjEns):
        print(f'Pixel {col},{row} - {QInj} fC')
        path_pattern = f"*{today.isoformat()}_Array_Test_Results/{scan_name}_Pixel_C{col}_R{row}_QInj_{QInj}"
        file_list = []
        for path, subdirs, files in os.walk(root):
            if not fnmatch(path, path_pattern): continue
            for name in files:
                pass
                if fnmatch(name, file_pattern):
                    file_list.append(os.path.join(path, name))
        total_files = len(file_list)
        for file_index, file_name in enumerate(file_list):
            with open(file_name) as infile:
                for line in infile:
                    text_list = line.split(',')
                    FPGA_state = text_list[0]
                    FPGA_data = int(text_list[3])
                    FPGA_triggerbit = int(text_list[5])
                    DAC = int(text_list[-1])
                    if DAC == -1: continue
                    try:
                        triggerbit_full_Scurve[row][col][QInj][DAC] = FPGA_triggerbit
                        data_full_Scurve[row][col][QInj][DAC] = FPGA_data
                    except:
                        print(f"Couldn't find DAC: {DAC} in file list")

### Plot Full Data S-Curve Scan

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*plot_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*7,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='r', ls='-', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='r', ls='-', lw=0.7)
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(data_full_Scurve[row][col][QInj].keys(), data_full_Scurve[row][col][QInj].values(), 'd-', color=colors[i], label=f"{QInj} fC Data",lw=0.5,markersize=1.5)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        plt.legend(loc="upper right")
        plt.yscale("log")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Full Data S-Curve")
        plt.tight_layout()
plt.savefig(fig_path+"/"+chip_figname+"_QInj_Full_Data_SCurveScan_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

### Plot Full Trigger Bit S-Curve Scan

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
col_list, row_list = zip(*plot_list)
u_cl = np.sort(np.unique(col_list))
u_rl = np.sort(np.unique(row_list))
fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*7,len(np.unique(u_rl))*5))
gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='r', ls='-', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='r', ls='-', lw=0.7)
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot(triggerbit_full_Scurve[row][col][QInj].keys(), triggerbit_full_Scurve[row][col][QInj].values(), '.--', color=colors[i], label=f"{QInj} fC TrigB",lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        plt.legend(loc="upper right")
        plt.yscale("log")
        plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Full Trigger Bit S-Curve")
        plt.tight_layout()
plt.savefig(fig_path+"/"+chip_figname+"_QInj_Full_TriggerBit_SCurveScan_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

# Run DAQ scanning by row to study multiple pixels TOT and TOA

### Define DAQ function

In [ ]:
def run_daq(timePerPixel, deadTime, triggerBitDelay, dirname):
    
    time_per_pixel = timePerPixel
    dead_time_per_pixel = deadTime
    trigger_bit_delay = triggerBitDelay
    total_scan_time = time_per_pixel + dead_time_per_pixel
    outname = dirname

    today = datetime.date.today()
    todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
    base_dir = Path(todaystr)
    base_dir.mkdir(exist_ok=True)

    parser = run_script.getOptionParser() 
    (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -t {int(total_scan_time)} -o {outname} -v -w -s 0x000C -p 0x000f -d 0x0800 --clear_fifo".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process'))
    process.start()

    IPC_queue.put('start L1A trigger bit')
    while not IPC_queue.empty():
        pass

    # delay = '000011'+format(trigger_bit_delay, '010b')
    # hex_delay = hex(int(delay, base=2))
    # IPC_queue.put(f'change delay {hex_delay}') 
    # while not IPC_queue.empty():
    #     pass

    time.sleep(time_per_pixel)
    IPC_queue.put('stop L1A trigger bit')

    time.sleep(1)
    IPC_queue.put('stop DAQ')
    while not IPC_queue.empty():
        pass

    IPC_queue.put('allow threads to exit')

    process.join()

### Actual DAQ run

In [ ]:
# Actual DAQ run
scan_name = chip_figname+"_VRef_TDC_DAQ"
# disable/default all pixels
print("Disabling all pixels")
i2c_conn.config_chips('00010000')

for i in range(15):

    DAQ_scan_list = list(zip(np.full(16, i), np.arange(16)))
    print(DAQ_scan_list)

    for row, col in DAQ_scan_list:
        pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)

        print(f"Enabling Pixel ({row},{col})")

        i2c_conn.pixel_decoded_register_write("Bypass_THCal", "0", pixel_connected_chip)               # Set to 0 to use BL from auto cal
        i2c_conn.pixel_decoded_register_write("QSel", format(0x0e, '05b'), pixel_connected_chip)       # Ensure we inject 0 fC of charge
        i2c_conn.pixel_decoded_register_write("TH_offset", format(0x0c, '06b'), pixel_connected_chip)  # Offset used to add to the auto BL for real triggering
        i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)             # ENable readout
        i2c_conn.pixel_decoded_register_write("QInjEn", "1", pixel_connected_chip)                     # ENable charge injection for the selected pixel
        i2c_conn.pixel_decoded_register_write("L1Adelay", format(0x01f5, '09b'), pixel_connected_chip) # Change L1A delay - circular buffer in ETROC2 pixel
        i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)                # Enable trigger path

    run_daq(10, 6, 485, f'{scan_name}_R{str(i)}_CX')

    for row, col in DAQ_scan_list:
        pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)

        print(f"Disabling Pixel ({row},{col})")

        i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)             # Disable readout
        i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)                     # Disable charge injection for the selected pixel
        i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)                # Disable trigger path
        i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)               # Set to 1 to use DAC
        i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)      # Set DAC manually to max

# Disconnect chip

In [ ]:
del i2c_conn

# Store BL, NW dataframe for later use

In [ ]:
outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (chip_name+tag+"_BaselineAt_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
BL_df.to_csv(outfile, index=False)

In [ ]:
note = 'PreTID'+chip_name
new_columns = {
    'note': f'{note}',
}

for col in new_columns:
    BL_df[col] = new_columns[col]

outdir = Path('../ETROC-History')
outfile = outdir / 'BaselineHistory.sqlite'

init_cmd = [
    'cd ' + str(outdir.resolve()),
    'git stash -u',
    'git pull',
]
end_cmd = [
    'cd ' + str(outdir.resolve()),
    'git add BaselineHistory.sqlite',
    'git commit -m "Added new history entry"',
    'git push',
    'git stash pop',
    'git stash clear',
]
init_cmd = [x + '\n' for x in init_cmd]
end_cmd  = [x + '\n' for x in end_cmd]

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

for cmd in init_cmd:
    p.stdin.write(cmd + "\n")
p.stdin.close()
p.wait()

print(p.stdout.read())

with sqlite3.connect(outfile) as sqlconn:
    BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

for cmd in end_cmd:
    p.stdin.write(cmd + "\n")
p.stdin.close()
p.wait()

p.stdin.close()

print(p.stdout.read())
